
# 0 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

import os

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 1 - Data loading

In [ ]:
# Input folder name
inputFolder = 'PAIR_003'
# Concatenate folder name with the path
inputPath = '../data/' + inputFolder + '/'
mesh1 = TriMesh(inputPath + 'target.off', area_normalize=True, center=False)
mesh2 = TriMesh(inputPath + 'source.off', area_normalize=True, center=False)

# 2- Visualization of eigen functions

In [ ]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=100, intrinsic=False, verbose=True);

# set up a figure twice as wide as it is tall
i = 0
d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, 0])
for i in range(1,10):
    d = mp.subplot(mesh1.vertlist, mesh1.facelist, c=mesh1.eigenvectors[:,i], s=[1, 10, i], data=d)

#d.save("myplottest")

# 3 - Computing the functional map

**Displaying data**

In [ ]:
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])
mp.subplot(mesh2.vertlist, mesh2.facelist, None, s=[2, 2, 1], data=d)

**Add correspondences**

In [ ]:
loadLandmark = True
if loadLandmark:
    landmarks = np.loadtxt(inputPath +'landmarks.txt',dtype=int)[:10]  # loading N landmarks
    display(landmarks)

    landmarks_mesh1 = TriMesh(mesh1.vertlist[landmarks[:,0]])
    landmarks_mesh2 = TriMesh(mesh2.vertlist[landmarks[:,1]])

    print('Retrieved matching faces from CT & RGBD pointclouds')
    
else:
     landmarks = []

#d = mp.subplot(mesh1.vertlist, mesh1.facelist, None, s=[2, 2, 0])

p1 = mp.plot(mesh1.vertlist, mesh1.facelist, None)
p2 = mp.plot(mesh2.vertlist, mesh2.facelist, None)

if loadLandmark:
    color = c=np.random.rand(*landmarks_mesh1.vertlist.shape)
    p1.add_points(landmarks_mesh1.vertlist, c = color, shading={"point_size": 0.3})
    p2.add_points(landmarks_mesh2.vertlist, c = color, shading={"point_size": 0.3})


**Generate functional map directly from landmarks**

In [ ]:
# Initialize vector 1 to 5
p2p_21 = np.array([0,1,2,3,4,5])
#p2p_21 = landmarks[:,1]
dim = len(landmarks[:,0])

resultingFM = FunctionalMapping(mesh1=mesh1, mesh2=mesh2)
resultingFM.mesh1.process(k=100, intrinsic=False, verbose=True);
resultingFM.mesh2.process(k=100, intrinsic=False, verbose=True);


print(f'p2p map  : {p2p_21} \n')
print(f'eigen vectors 1 : {resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim]} \n')
print(f'eigen vectors 2 : {resultingFM.mesh2.eigenvectors[landmarks[:,1], :dim]} \n')
print(f'dim : {dim} \n') 

#test = pyFM.spectral.convert.p2p_to_FM(p2p_21, mesh1, mesh2, use_adj=False, subsample=None, use_ANN=False, n_jobs=1)
import pyFM.spectral.convert
#test = pyFM.spectral.convert.mesh_p2p_to_FM(p2p_21, mesh1, mesh2, dim, [landmarks[:,0], landmarks[:,1]])
test = pyFM.spectral.convert.p2p_to_FM(p2p_21, resultingFM.mesh1.eigenvectors[landmarks[:,0], :dim+9], resultingFM.mesh2.eigenvectors[landmarks[:,1], :dim+9])

#mesh_p2p_to_FM(p2p_21, mesh1, mesh2, dims=None, subsample=None):
#p2p_to_FM(p2p_21, evects1, evects2, A2=None):

print(f'FM : {test} \n')

# Resuling FM
resultingFM.FM = test
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)



In [ ]:
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=12, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=12, intrinsic=False, verbose=True);

#p2p_21 = resultingFM.get_p2p(n_jobs=1)
#cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
#double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=9, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=9, intrinsic=False, verbose=True);

#p2p_21 = resultingFM.get_p2p(n_jobs=1)
#cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
#double_plot(mesh1,mesh2,cmap1,cmap2)

In [ ]:
p2p_21 = resultingFM.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

#resultingFM.mesh1.process(k=9, intrinsic=False, verbose=True);
#resultingFM.mesh2.process(k=9, intrinsic=False, verbose=True);



resultingFM.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
resultingFM._FM_base = test

resultingFM.zoomout_refine(nit=10, step = 2, verbose=True)
print(resultingFM.FM.shape)

p2p_21_zo = resultingFM.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [ ]:
# check if data_out folder exists, if not create it
if not os.path.exists('data_out'):
    os.makedirs('data_out')

# export point to point mappings if they exist

# zoomout refined point to point mapping
if 'p2p_21_zo' in locals():
    # open file in write mode
    with open(r'data_out/p2p_zo.txt', 'w') as f_export_zo:
        for item in p2p_21_zo:
            # write each item on a new line
            f_export_zo.write("%s\n" % item)
        print('Exported zoomout refined point to point mapping')